In [1]:
import pandas as pd
from fundamentos import (
                          CalculoWACC, 
                          VariacaoReceita, 
                          ValuationModoloGordon, 
                          OutrosAtivosNaoOperacionais,
                          PassivoTotalMenosDivida,
                          NecessidadeCapitalGiro,
                          IndicadoresFinanceiros,
                          ValuationFluxoCaixaDescontado)

from dotenv import load_dotenv
import ipeadatapy as ip
from datetime import datetime
import pandas as pd
import yfinance as yf
import sidrapy
import warnings
warnings.filterwarnings('ignore')
load_dotenv()


False

# Valuation modolo gordon

In [ ]:
# Valuation modolo gordon
acoes = pd.read_csv("C:/Users/jefer/Downloads/setor.csv")['tic'].to_list()
lista_valuation = []
for acao in acoes:
    acao_sa = acao + ".SA"
    valu = ValuationModoloGordon(acao_sa)
    try:
        dicionario = valu.preco_acao()
        lista_valuation.append(dicionario)
    except KeyError:
        print("Erro ao obter dados da ação:", acao_sa)
    #print(dicionario)
    
valuation_acoes = pd.DataFrame(lista_valuation)
valuation_acoes.sort_values(by='diferenca', ascending=False).head(40)

# Valuation fluxo de caixa descontado

In [ ]:
import warnings
warnings.filterwarnings('ignore')
acoes = pd.read_csv("C:/Users/jefer/Downloads/setor.csv")['tic'].to_list()
for acao in acoes:
    try:
        wac = CalculoWACC(ticker=f'{acao}.SA')
        print(f'Custo Capital Proprio da acao {acao}:',wac.wacc())
    except KeyError:
        print(f'Erro ao obter dados da acao {acao}')
    except ValueError:
        print(f'Erro ao obter dados da acao {acao}')

In [ ]:
acoes = pd.read_csv("C:/Users/jefer/Downloads/setor.csv")['tic'].to_list()
for acao in acoes:
    try:
        vari = VariacaoReceita(ticker=f'{acao}.SA')
        print(f'Custo Capital Proprio da acao {acao}:',vari.receita_crescimento_metricas())
    except KeyError:
        print(f'Erro ao obter dados da acao {acao}')
    except ValueError:
        print(f'Erro ao obter dados da acao {acao}')
    except ConnectionError:
        print(f'Erro ao obter dados da acao {acao}')

In [2]:
ind = IndicadoresFinanceiros(ticker='AZEV3.SA')
indicadores_petr = ind.todos_indicadores()
indicadores_petr

[*********************100%***********************]  1 of 1 completed


{'margemebit': -0.188,
 'ultimareceita': 442596000.0,
 'variacaoreceita': {'mean_deflacionada': 1.8544798993495084,
  'median_deflacionada': 1.7348590789166258,
  'mean_normal': 2.054389064389649,
  'median_normal': 2.009985902255639},
 'depreciacaocapex': 0.1,
 'capexreceita': 0.1,
 'wacc': 0.166,
 'quantidadeacoes': 187167464.0,
 'dividatotal': 39103000.0,
 'caixa': 2090000.0,
 'outrosativos': 360930000.0,
 'passivosmenosdivida': 367763000.0,
 'necessidadecapitalgiro': -104186000.0,
 'percentualimposto': 0.5714}

In [2]:
ind = IndicadoresFinanceiros(ticker='PETR4.SA')
pd.options.display.float_format = '{:.2f}'.format
indicadores_petr = ind.todos_indicadores()
print(indicadores_petr)
valuation = ValuationFluxoCaixaDescontado(
            receita_ano=indicadores_petr['ultimareceita'],
            porcenta_crescimento_receita=indicadores_petr['variacaoreceita']['median_deflacionada'] if 'median_deflacionada' in indicadores_petr['variacaoreceita'].keys() else indicadores_petr['variacaoreceita']['median_normal'],
            margem_ebit=indicadores_petr['margemebit'],
            imposto_porcentagem=indicadores_petr['percentualimposto'],
            depreciacao_capex=indicadores_petr['depreciacaocapex'],
            capex_da_receita=indicadores_petr['capexreceita'],
            wacc=indicadores_petr['wacc'],
            numero_de_acoes=indicadores_petr['quantidadeacoes'],
            divida=indicadores_petr['dividatotal'],
            disponivel=indicadores_petr['caixa'],
            ativos_nao_operacionais=indicadores_petr['outrosativos'],
            passivos_circulantes=indicadores_petr['passivosmenosdivida'],
            necessidade_capital_de_giro=indicadores_petr['necessidadecapitalgiro'],
            anos_projecao=5, 
            taxa_crecimento_perpetuidade=0.014,
            calculo_necessidade_capital_de_giro=False)
print(valuation.calcular_valuation())

[*********************100%***********************]  1 of 1 completed


{'margemebit': 0.3882, 'ultimareceita': 102409000000.0, 'variacaoreceita': {'mean_deflacionada': 0.20294717006024565, 'median_deflacionada': 0.40129820010037975, 'mean_normal': 0.28975841631631644, 'median_normal': 0.4824333658861921}, 'depreciacaocapex': 1.6143, 'capexreceita': 0.0932, 'wacc': 0.164, 'quantidadeacoes': 13044496930.0, 'dividatotal': 62600000000.0, 'caixa': 15546000000.0, 'outrosativos': 170536000000.0, 'passivosmenosdivida': 75492000000.0, 'necessidadecapitalgiro': -1415000000.0, 'percentualimposto': 0.2666}
Valuation para 5 anos
Anos 5.00
Valor presente do fluxo de caixa: R$ 1080775988712.18
Fluxo de caixa ajustado: R$ 1191365988712.18
Valor por ação: R$ 91.33
   data     receita_ano        ebit_ano    imposto_ano      capex_ano  \
0  2025 143505547374.08  55708853490.62 14851980340.60 13374717015.26   
1  2026 201094065239.72  78064716126.06 20812053319.21 18741966880.34   
2  2027 281792751671.29 109391946198.79 29163892856.60 26263084455.76   
3  2028 394875675718.

In [ ]:
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.2f}'.format
acao = pd.read_csv("C:/Users/jefer/Downloads/setor.csv")['tic'].to_list()

for acao in acao:
    print(f'Valuation Fluxo de Caixa Descontado da acao {acao}')
    try:
        ind = IndicadoresFinanceiros(ticker=f'{acao}.SA')
        indicadores_petr = ind.todos_indicadores()
        print(indicadores_petr)
        valuation = ValuationFluxoCaixaDescontado(
            receita_ano=indicadores_petr['ultimareceita'],
            porcenta_crescimento_receita=indicadores_petr['variacaoreceita']['median_deflacionada'] if 'median_deflacionada' in indicadores_petr['variacaoreceita'].keys() else indicadores_petr['variacaoreceita']['median_normal'],
            margem_ebit=indicadores_petr['margemebit'],
            imposto_porcentagem=indicadores_petr['percentualimposto'],
            depreciacao_capex=indicadores_petr['depreciacaocapex'],
            capex_da_receita=indicadores_petr['capexreceita'],
            wacc=indicadores_petr['wacc'],
            numero_de_acoes=indicadores_petr['quantidadeacoes'],
            divida=indicadores_petr['dividatotal'],
            disponivel=indicadores_petr['caixa'],
            ativos_nao_operacionais=indicadores_petr['outrosativos'],
            passivos_circulantes=indicadores_petr['passivosmenosdivida'],
            necessidade_capital_de_giro=indicadores_petr['necessidadecapitalgiro'],
            anos_projecao=5, 
            taxa_crecimento_perpetuidade=0.014,
            calculo_necessidade_capital_de_giro=False)
        print(valuation.calcular_valuation())
    except:
        print(f'Erro ao obter dados da acao {acao}')

# Valuation completo

In [31]:
acoes= pd.read_csv("C:/Users/jefer/Downloads/setor.csv")['tic'].to_list()

In [32]:
acoes[:10]

['AALR3',
 'ABEV3',
 'AFLT3',
 'AGRO3',
 'AGXY3',
 'AHEB3',
 'ALPA3',
 'ALPA4',
 'ALPK3',
 'ALLD3']

In [49]:
pd.read_csv("dados/setor.csv")['tic'].to_list()

['AALR3',
 'ABEV3',
 'AFLT3',
 'AGRO3',
 'AGXY3',
 'AHEB3',
 'ALPA3',
 'ALPA4',
 'ALPK3',
 'ALLD3',
 'ALOS3',
 'AMBP3',
 'AMAR3',
 'AMER3',
 'ANIM3',
 'ASAI3',
 'AURE3',
 'AVLL3',
 'AZEV3',
 'AZUL4',
 'AZZA3',
 'BALM4',
 'BAUH4',
 'BAZA3',
 'BBAS3',
 'BBSE3',
 'BBDC3',
 'BBDC4',
 'BGIP3',
 'BGIP4',
 'BIOM3',
 'BLAU3',
 'BMEB3',
 'BMEB4',
 'BMKS3',
 'BMIN3',
 'BMIN4',
 'BMOB3',
 'BPAN4',
 'BPAC11',
 'BPAC3',
 'BPAC5',
 'BRAP3',
 'BRAP4',
 'BRAV3',
 'BRBI11',
 'BRFS3',
 'BRKM3',
 'BRKM5',
 'BRST3',
 'BRSR3',
 'BRSR6',
 'BSLI3',
 'BSLI4',
 'CAMB3',
 'CAML3',
 'CASH3',
 'CCRO3',
 'CEAB3',
 'CEBR6',
 'CEDO3',
 'CEDO4',
 'CGAS3',
 'CGAS5',
 'CLSA3',
 'CMIG3',
 'CMIG4',
 'COGN3',
 'CRFB3',
 'CSAN3',
 'CSMG3',
 'CSNA3',
 'CSUD3',
 'CYRE3',
 'DIRR3',
 'DXCO3',
 'EALT3',
 'EALT4',
 'ECOR3',
 'EGIE3',
 'ELET3',
 'ELET6',
 'EMBR3',
 'ENEV3',
 'ENGI11',
 'ENGI3',
 'ENGI4',
 'EQTL3',
 'ESPA3',
 'EZTC3',
 'FESA3',
 'FESA4',
 'FIQE3',
 'FLRY3',
 'FRAS3',
 'GGBR3',
 'GGBR4',
 'GOAU3',
 'GOAU4',
 'GOLL4

In [ ]:
acoes= pd.read_csv("dados/setor.csv")['tic'].to_list()
acoes = acoes[:5]
data_valuation = {
    'acao': [],
    'valor_atual':[],
    'preco_gordon': [],
    'preco_fluxo': [],
    'diferenca_gordon': [],
    'diferenca_fluxo': []
}
for acao in acoes:
    print('-'*10, acao, '-'*10)
    try:
        valu = ValuationModoloGordon(f'{acao}.SA')
        preco_gordon = valu.preco_acao()

        ind = IndicadoresFinanceiros(ticker=f'{acao}.SA')
        indicadores = ind.todos_indicadores()

        valuation_fluxo = ValuationFluxoCaixaDescontado(
                    receita_ano=indicadores['ultimareceita'],
                    porcenta_crescimento_receita=indicadores['variacaoreceita']['median_deflacionada'] if 'median_deflacionada' in indicadores['variacaoreceita'].keys() else indicadores['variacaoreceita']['median_normal'],
                    margem_ebit=indicadores['margemebit'],
                    imposto_porcentagem=indicadores['percentualimposto'],
                    depreciacao_capex=indicadores['depreciacaocapex'],
                    capex_da_receita=indicadores['capexreceita'],
                    wacc=indicadores['wacc'],
                    numero_de_acoes=indicadores['quantidadeacoes'],
                    divida=indicadores['dividatotal'],
                    disponivel=indicadores['caixa'],
                    ativos_nao_operacionais=indicadores['outrosativos'],
                    passivos_circulantes=indicadores['passivosmenosdivida'],
                    necessidade_capital_de_giro=indicadores['necessidadecapitalgiro'],
                    anos_projecao=5, 
                    taxa_crecimento_perpetuidade=0.014,
                    calculo_necessidade_capital_de_giro=False)

        _ , valor_fluxo = valuation_fluxo.calcular_valuation()

        yaho = yf.Ticker(f'{acao}.SA')
        valor_acao_atual = round(yaho.history(period="10Y", interval="1d")['Close'][-1],2)
        
        data_valuation['acao'].append(acao)
        data_valuation['valor_atual'].append(valor_acao_atual)
        data_valuation['preco_gordon'].append(preco_gordon['valuation_acao'])
        data_valuation['preco_fluxo'].append(valor_fluxo['valor_por_acao'])
        data_valuation['diferenca_gordon'].append(round((preco_gordon['valuation_acao'] - valor_acao_atual)/valor_acao_atual,2))
        data_valuation['diferenca_fluxo'].append(round((valor_fluxo['valor_por_acao'] - valor_acao_atual)/valor_acao_atual,2))
    except:
        print(f'Erro ao obter dados da acao {acao}')
data_valores = pd.DataFrame(data_valuation)

In [51]:
data_valores.sort_values(by=['diferenca_fluxo', 'diferenca_gordon'], ascending=False).to_csv('dados/valores_valuations_acoes.csv', index=False)




In [ ]:
data_valuation['acao'].append(acao)
data_valuation['valor_atual'].append(valor_acao_atual)
data_valuation['preco_gordon'].append(preco_gordon['valuation_acao'])
data_valuation['preco_fluxo'].append(valor_fluxo['valor_por_acao'])
